<a href="https://colab.research.google.com/github/CloneTerial/Ai-/blob/main/learn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [78]:

import pandas as pd
import re

df = pd.read_csv('conversation_dataset.csv')
df.head()

,question,answer
0,Apa arti dari abadi?,kekal; tidak berkesudahan: di dunia ini tidak ...
1,Apa arti dari abaktinal?,istilah biologi berkenaan dengan sisi tubuh ya...
2,Apa arti dari abang?,merah
3,Apa arti dari abjadiah?,berdasarkan abjad
4,Apa arti dari abnormal?,tidak sesuai dengan keadaan yang biasa; mempun...


In [ ]:
# Install dependencies
!pip install transformers
!pip install datasets
!pip install tensorflow

In [54]:
import pandas as pd

# Memuat dataset percakapan yang telah dibuat
input_file = 'conversation_dataset.csv'  # Ganti dengan nama file kamu yang sudah dibuat
df = pd.read_csv(input_file)

# Menggabungkan kolom 'question' dan 'answer' menjadi satu kolom teks
train_texts = df['question'] + " " + df['answer']

# Menyimpan dataset sebagai file teks untuk fine-tuning
with open('train_data.txt', 'w') as f:
    for text in train_texts:
        f.write(f"{text}\n")

In [63]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from datasets import load_dataset
import tensorflow as tf

# Memuat tokenizer dan model GPT-2
model_name = "gpt2"  # Kamu bisa menggunakan 'gpt2-medium', 'gpt2-large', atau 'gpt2-xl' jika memerlukan model yang lebih besar
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Pastikan tokenizer dapat menangani panjang input yang lebih panjang
tokenizer.pad_token = tokenizer.eos_token

# Memuat dataset dari file teks yang telah dibuat
def encode_examples(example):
    # Tokenisasi teks dan membuat label sama dengan input (GPT-2 akan memprediksi token berikutnya)
    encoding = tokenizer(example['text'], truncation=True, padding='max_length', max_length=512)
    encoding['labels'] = encoding['input_ids']  # Menambahkan kolom 'labels' yang sama dengan 'input_ids'
    return encoding

train_dataset = load_dataset('text', data_files={'train': 'train_data.txt'}, split='train')
train_dataset = train_dataset.map(lambda x: {'text': x['text']})
train_dataset = train_dataset.map(encode_examples, batched=True)

# Menampilkan beberapa contoh data untuk memastikan formatnya benar
print(train_dataset[0])

Map:   0%|          | 0/3596 [00:00<?, ? examples/s]

{'text': 'Apa arti dari abadi? kekal; tidak berkesudahan: di dunia ini tidak ada yang abadi', 'input_ids': [32, 8957, 1242, 72, 288, 2743, 450, 9189, 30, 885, 74, 282, 26, 29770, 461, 18157, 5209, 463, 19210, 25, 2566, 12574, 544, 287, 72, 29770, 461, 512, 64, 331, 648, 450, 9189, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 5025

In [64]:
from transformers import Trainer, TrainingArguments

# Menyiapkan argumen pelatihan
training_args = TrainingArguments(
    output_dir="./gpt2-finetuned",  # Tempat menyimpan model yang dilatih
    num_train_epochs=3,  # Jumlah epoch, bisa disesuaikan
    per_device_train_batch_size=2,  # Ukuran batch per perangkat
    gradient_accumulation_steps=16,  # Mengakumulasi gradient untuk batch yang lebih besar
    logging_dir='./logs',  # Tempat untuk log
    logging_steps=10,  # Setiap berapa langkah log dilakukan
    save_steps=10_000,  # Menyimpan model setiap 10.000 langkah
    save_total_limit=2,  # Hanya menyimpan 2 model terakhir
)

# Menyiapkan Trainer
trainer = Trainer(
    model=model,  # Model yang dilatih
    args=training_args,  # Argumen pelatihan
    train_dataset=train_dataset,  # Dataset pelatihan
    tokenizer=tokenizer  # Tokenizer yang digunakan
)

# Melatih model
trainer.train()

<ipython-input-64-ca3ae168f7d4>:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
10,1.983300
20,0.508500
30,0.456400
40,0.398000
50,0.353000
60,0.357000
70,0.317200
80,0.344900
90,0.338200
100,0.332000


Step,Training Loss
10,1.983300
20,0.508500
30,0.456400
40,0.398000
50,0.353000
60,0.357000
70,0.317200
80,0.344900
90,0.338200
100,0.332000


TrainOutput(global_step=336, training_loss=0.3716816692834809, metrics={'train_runtime': 1633.4877, 'train_samples_per_second': 6.604, 'train_steps_per_second': 0.206, 'total_flos': 2809411928064000.0, 'train_loss': 0.3716816692834809, 'epoch': 2.9899888765294773})

In [65]:
# Menyimpan model yang telah dilatih
model.save_pretrained("./gpt2-finetuned")
tokenizer.save_pretrained("./gpt2-finetuned")

('./gpt2-finetuned/tokenizer_config.json',
 './gpt2-finetuned/special_tokens_map.json',
 './gpt2-finetuned/vocab.json',
 './gpt2-finetuned/merges.txt',
 './gpt2-finetuned/added_tokens.json')

In [66]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Memuat model dan tokenizer yang telah dilatih
model = GPT2LMHeadModel.from_pretrained("./gpt2-finetuned")
tokenizer = GPT2Tokenizer.from_pretrained("./gpt2-finetuned")

# Pastikan tokenizer dapat menangani panjang input yang lebih panjang
tokenizer.pad_token = tokenizer.eos_token

In [72]:

inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)
attention_mask = inputs['attention_mask']
input_ids = inputs['input_ids']

outputs = model.generate(input_ids, attention_mask=attention_mask, max_length=100)
def generate_response(input_text):
    # Mengonversi input teks menjadi token yang bisa diproses oleh GPT-2
    inputs = tokenizer.encode(input_text, return_tensors="pt")

    # Menghasilkan output dari model
    outputs = model.generate(
        inputs,
        max_length=100,  # Panjang respons yang dihasilkan
        num_return_sequences=1,  # Jumlah respons yang dihasilkan
        no_repeat_ngram_size=2,  # Mencegah pengulangan kata
        top_p=0.85,  # Pengaturan untuk mengatur kemungkinan kata berikutnya
        top_k=50,  # Pengaturan untuk top-k sampling
        do_sample=True,  # Sampling alih-alih menggunakan argmax
    )

    # Mengubah output token menjadi teks yang dapat dibaca
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# Uji model dengan pertanyaan
input_text = "Apa arti dari abadi?"
print(generate_response(input_text))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Apa arti dari abadi? berhubungan dengan berabadi: ia membuat berhadap karup banyak keluruh berturahan kita


In [ ]:
while True:
    user_input = input("Kamu: ")
    if user_input.lower() == 'exit':
        break
    response = generate_response(user_input)
    print(f"Bot: {response}")

In [ ]:
# Mengompres folder menjadi file ZIP
!zip -r /content/L.zip /content/gpt2-finetuned